In [90]:
import json

def load_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
        angiography_importance = []
        for case in data:
            for key in case.keys():
                if "Importance of coronary angiography" in key:
                    
                    value = case[key].strip('",')
                    angiography_importance.append(int(value))
                    break
    return angiography_importance


angiography_importance_male_prompt_1 = load_data('Outputs/CAD_results_male.json')
angiography_importance_female_prompt_1 = load_data('Outputs/CAD_results_female.json')
angiography_importance_male_prompt2= load_data('Outputs/CAD_results_male_best4.json')
angiography_importance_female_prompt2= load_data('Outputs/CAD_results_female_best4.json')
angiography_importance_male_prompt3= load_data('Outputs/CAD_results_male_best_n.json')
angiography_importance_female_prompt3= load_data ('Outputs/CAD_results_female_best_n.json')
angiography_importance_male_prompt4= load_data ('Outputs/CAD_results_male1.json')
angiography_importance_female_prompt4= load_data('outputs/CAD_results_female1.json')
angiography_importance_male_prompt5= load_data('Outputs/CAD_results_male3.json')
angiography_importance_female_prompt5= load_data('Outputs/CAD_results_female3.json')




In [91]:
import json
import pandas as pd
from scipy import stats


# Combine data into a DataFrame
data = {
    'values': angiography_importance_male_prompt_1 + angiography_importance_female_prompt_1 + 
              angiography_importance_male_prompt2 + angiography_importance_female_prompt2 + 
              angiography_importance_male_prompt3 + angiography_importance_female_prompt3 + 
              angiography_importance_male_prompt4 + angiography_importance_female_prompt4 + 
              angiography_importance_male_prompt5 + angiography_importance_female_prompt5,
    'prompt': ['Male Prompt 1'] * len(angiography_importance_male_prompt_1) + 
              ['Female Prompt 1'] * len(angiography_importance_female_prompt_1) + 
              ['Male Prompt 2'] * len(angiography_importance_male_prompt2) + 
              ['Female Prompt 2'] * len(angiography_importance_female_prompt2) + 
              ['Male Prompt 3'] * len(angiography_importance_male_prompt3) + 
              ['Female Prompt 3'] * len(angiography_importance_female_prompt3) + 
              ['Male Prompt 4'] * len(angiography_importance_male_prompt4) + 
              ['Female Prompt 4'] * len(angiography_importance_female_prompt4) + 
              ['Male Prompt 5'] * len(angiography_importance_male_prompt5) + 
              ['Female Prompt 5'] * len(angiography_importance_female_prompt5),
    'gender': ['Male'] * len(angiography_importance_male_prompt_1) + 
              ['Female'] * len(angiography_importance_female_prompt_1) + 
              ['Male'] * len(angiography_importance_male_prompt2) + 
              ['Female'] * len(angiography_importance_female_prompt2) + 
              ['Male'] * len(angiography_importance_male_prompt3) + 
              ['Female'] * len(angiography_importance_female_prompt3) + 
              ['Male'] * len(angiography_importance_male_prompt4) + 
              ['Female'] * len(angiography_importance_female_prompt4) + 
              ['Male'] * len(angiography_importance_male_prompt5) + 
              ['Female'] * len(angiography_importance_female_prompt5) 
}


df = pd.DataFrame(data)

# Perform ANOVA for Male
f_value_male, p_value_male = stats.f_oneway(*[df[(df['prompt'] == prompt) & (df['gender'] == 'Male')]['values'] for prompt in df['prompt'].unique() if 'Male' in prompt])
print("ANOVA for Male - F-value:", f_value_male, "P-value:", p_value_male)

# Perform ANOVA for Female
f_value_female, p_value_female = stats.f_oneway(*[df[(df['prompt'] == prompt) & (df['gender'] == 'Female')]['values'] for prompt in df['prompt'].unique() if 'Female' in prompt])
print("ANOVA for Female - F-value:", f_value_female, "P-value:", p_value_female)

 

ANOVA for Male - F-value: 20.479928440527086 P-value: 1.2495789409599345e-15
ANOVA for Female - F-value: 12.170374747470383 P-value: 1.917010331772426e-09


In [92]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
# Specific titles for each prompt
specific_titles = {
    'Male Prompt 1': 'Avoid bias promp (Male)',
    'Female Prompt 1': 'Avoid bias prompt (Female)',
    'Male Prompt 2': 'Best objective doctor and pleasant male patient (Male)',
    'Female Prompt 2': 'Best objective doctor and pleasant male patient (Female)',
    'Male Prompt 3': 'Best objective doctor (Male)',
    'Female Prompt 3': 'Best objective doctor (Female)',
    'Male Prompt 4': 'Baseline neutral prompt (Male)',
    'Female Prompt 4': 'Baseline neutral prompt (Female)',
    'Male Prompt 5': 'Neutral and pleasant male patient (Male)',
    'Female Prompt 5': 'Neutral and pleasant male patient (Female)'
}

df['prompt'] = df['prompt'].map(specific_titles)

# Separate data for ANOVA for Male
male_data = [df[(df['prompt'] == prompt) & (df['gender'] == 'Male')]['values'] for prompt in specific_titles.values() if 'Male' in prompt]

# Perform ANOVA for Male
f_value_male, p_value_male = stats.f_oneway(*male_data)
print("ANOVA for Male - F-value:", f_value_male, "P-value:", p_value_male)

# Separate data for ANOVA for Female
female_data = [df[(df['prompt'] == prompt) & (df['gender'] == 'Female')]['values'] for prompt in specific_titles.values() if 'Female' in prompt]

# Perform ANOVA for Female
f_value_female, p_value_female = stats.f_oneway(*female_data)
print("ANOVA for Female - F-value:", f_value_female, "P-value:", p_value_female)




ANOVA for Male - F-value: 20.479928440527086 P-value: 1.2495789409599345e-15
ANOVA for Female - F-value: 12.170374747470383 P-value: 1.917010331772426e-09


In [93]:
anova_results = pd.DataFrame({
    'Group': ['Male', 'Female'],
    'F-value': [f_value_male, f_value_female],
    'P-value': [p_value_male, p_value_female]
})

display(anova_results)

,Group,F-value,P-value
0,Male,20.479928,1.249579e-15
1,Female,12.170375,1.917010e-09


In [94]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Tukey HSD test for Male
tukey_male = pairwise_tukeyhsd(endog=df[df['gender'] == 'Male']['values'],
                               groups=df[df['gender'] == 'Male']['prompt'],
                               alpha=0.05)

# Tukey HSD test for Female
tukey_female = pairwise_tukeyhsd(endog=df[df['gender'] == 'Female']['values'],
                                 groups=df[df['gender'] == 'Female']['prompt'],
                                 alpha=0.05)



In [95]:
# Convert Tukey HSD test results for Male and Female into DataFrames
tukey_male_df = pd.DataFrame(data=tukey_male.summary().data[1:], columns=tukey_male.summary().data[0])
tukey_female_df = pd.DataFrame(data=tukey_female.summary().data[1:], columns=tukey_female.summary().data[0])


In [96]:

display (tukey_male_df)
display (tukey_female_df)



,group1,group2,meandiff,p-adj,lower,upper,reject
0,Avoid bias promp (Male),Baseline neutral prompt (Male),0.48,0.0005,0.1573,0.8027,True
1,Avoid bias promp (Male),Best objective doctor (Male),0.89,0.0000,0.5673,1.2127,True
2,Avoid bias promp (Male),Best objective doctor and pleasant male patien...,0.92,0.0000,0.5973,1.2427,True
3,Avoid bias promp (Male),Neutral and pleasant male patient (Male),0.69,0.0000,0.3673,1.0127,True
4,Baseline neutral prompt (Male),Best objective doctor (Male),0.41,0.0050,0.0873,0.7327,True
5,Baseline neutral prompt (Male),Best objective doctor and pleasant male patien...,0.44,0.0020,0.1173,0.7627,True
6,Baseline neutral prompt (Male),Neutral and pleasant male patient (Male),0.21,0.3853,-0.1127,0.5327,False
7,Best objective doctor (Male),Best objective doctor and pleasant male patien...,0.03,0.9991,-0.2927,0.3527,False
8,Best objective doctor (Male),Neutral and pleasant male patient (Male),-0.20,0.4367,-0.5227,0.1227,False
9,Best objective doctor and pleasant male patien...,Neutral and pleasant male patient (Male),-0.23,0.2916,-0.5527,0.0927,False


,group1,group2,meandiff,p-adj,lower,upper,reject
0,Avoid bias prompt (Female),Baseline neutral prompt (Female),0.6896,0.0001,0.2755,1.1037,True
1,Avoid bias prompt (Female),Best objective doctor (Female),0.8996,0.0000,0.4855,1.3137,True
2,Avoid bias prompt (Female),Best objective doctor and pleasant male patien...,0.8496,0.0000,0.4355,1.2637,True
3,Avoid bias prompt (Female),Neutral and pleasant male patient (Female),0.8296,0.0000,0.4155,1.2437,True
4,Baseline neutral prompt (Female),Best objective doctor (Female),0.2100,0.6332,-0.2031,0.6231,False
5,Baseline neutral prompt (Female),Best objective doctor and pleasant male patien...,0.1600,0.8266,-0.2531,0.5731,False
6,Baseline neutral prompt (Female),Neutral and pleasant male patient (Female),0.1400,0.8860,-0.2731,0.5531,False
7,Best objective doctor (Female),Best objective doctor and pleasant male patien...,-0.0500,0.9974,-0.4631,0.3631,False
8,Best objective doctor (Female),Neutral and pleasant male patient (Female),-0.0700,0.9905,-0.4831,0.3431,False
9,Best objective doctor and pleasant male patien...,Neutral and pleasant male patient (Female),-0.0200,0.9999,-0.4331,0.3931,False
